<a href="https://colab.research.google.com/github/dame-cell/Gaja/blob/main/training_the_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# steps followed for the full-fine tuning:

1. First we import and install the dependencies
2. second we load the configuration which makes life easier
3. mount goggle drive so that we can store our checkpoints
4. downloading the base-model using unsloth and in 4-bit
5. Get the peft-configuration
6. load the datasets and also pre-precoess it a bit
7. and finally truly starting the training



In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install "unsloth[colab_ampere] @ git+https://github.com/unslothai/unsloth.git"
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"
pass

!pip install "git+https://github.com/huggingface/transformers.git" # Native 4bit loading works!

In [ ]:
import yaml

# Specify the path to your YAML configuration file
config_file_path = "config.yaml"

# Load the YAML file
with open(config_file_path, "r") as config_file:
    config = yaml.safe_load(config_file)

# Access all configuration parameters
max_seq_length = config.get('max_seq_length')
dtype = config.get('dtype')
load_in_4bit = config.get('load_in_4bit')
model_name = config.get('model_name')
target_modules = config.get('target_modules')
ra_alpha = config.get('ra_alpha')
lora_dropout = config.get('lora_dropout')
bias = config.get('bias')
use_gradient_checkpointing = config.get('use_gradient_checkpointing')
random_state = config.get('random_state')
use_rslora = config.get('use_rslora')
loftq_config = config.get('loftq_config')
dataset_text_field = config.get('dataset_text_field')
dataset_num_proc = config.get('ataset_num_proc')  # Typo in the original config, fix it to 'dataset_num_proc'
packing = config.get('packing')
per_device_train_batch_size = config.get('per_device_train_batch_size')
gradient_accumulation_steps = config.get('gradient_accumulation_steps')
warmup_steps = config.get('warmup_steps')
hub_strategy = config.get('hub_strategy')
num_train_epochs = config.get('num_train_epochs')
push_to_hub = config.get('push_to_hub')
push_to_hub_model_id = config.get('push_to_hub_model_id')
learning_rate = config.get('learning_rate')
resume_from_checkpoint = config.get('resume_from_checkpoint')

logging_steps = config.get('logging_steps')
optim = config.get('optim')
weight_decay = config.get('weight_decay')
save_total_limit = config.get('save_total_limit')
save_steps = config.get('save_steps')
lr_scheduler_type = config.get('lr_scheduler_type')
seed = config.get('seed')

FileNotFoundError: [Errno 2] No such file or directory: 'config.yaml'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from unsloth import FastLanguageModel
import torch



model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "sarvamai/OpenHathi-7B-Hi-v0.1-Base",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

/usr/local/lib/python3.10/dist-packages/unsloth/__init__.py:66: UserWarning: Running `ldconfig /usr/lib64-nvidia` to link CUDA.
  warnings.warn(


config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.2
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/936 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/968k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = target_modules,
    lora_alpha = 16,
    lora_dropout = lora_dropout,
    bias = bias,
    use_gradient_checkpointing = use_gradient_checkpointing,
    random_state = random_state,
    use_rslora = use_rslora,
    loftq_config = loftq_config,
)

Unsloth 2024.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("damerajee/insutrct-vls",split='train')

Generating train split:   0%|          | 0/55215 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['Unnamed: 0', 'instruction', 'input', 'output'],
    num_rows: 55215
})

In [ ]:
data= dataset.remove_columns('Unnamed: 0')

In [ ]:
data

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 55215
})

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:



EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = data.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/55215 [00:00<?, ? examples/s]

In [ ]:
dataset['text'][0]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n### Instruction:\nNone\n\n### Input:\nमुझे अपनी ऑनलाइन साइट के लिए एक विवरण चाहिए जो उन कपड़ों का वर्णन करता है जिन्हें मैं बेचने की कोशिश कर रहा हूं। वे प्रवाहित और एक तरह से जादुई हैं। 3 रंग हैंः हल्का नीला, गुलाबी और सफेद। पोशाक घुटने के ठीक ऊपर से टकराती है और इसमें छोटी बाजू होती है। मुझे कुछ ऐसा चाहिए जो वास्तव में इस उत्पाद को बेचने में मेरी मदद करे। अन्य जानकारी यह है कि इसका आकार xs से x-बड़ा होता है। इसे कुछ वाक्यों तक रखने की कोशिश करें (<5 अधिमानतः)।\n\n### Response:\nएक ऐसी बहती पोशाक की तलाश है जो आपको हल्का और हवादार महसूस कराए? आगे नहीं देखें तीन स्वप्निल रंगों में से चुनने के लिए, सभी के लिए कुछ न कुछ है। यह पोशाक घुटने के ठीक ऊपर से टकराती है और एक जादुई प्रभाव पैदा करने के लिए आपके चारों ओर तैरती है जिसे हर कोई पसंद करेगा। आकारों में अतिरिक्त छोटे से लेकर अतिरिक्त बड़े शामिल हैं।</s>'

In [ ]:
dataset = dataset.shuffle(seed=42)

In [ ]:
output_dir = "output-path"

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = dataset_text_field,
    max_seq_length = max_seq_length,
    dataset_num_proc = dataset_num_proc,
    packing = packing, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = per_device_train_batch_size,
        gradient_accumulation_steps = gradient_accumulation_steps,
        warmup_steps = warmup_steps,
        hub_strategy=hub_strategy,
        num_train_epochs=num_train_epochs,
        push_to_hub=push_to_hub,
        push_to_hub_model_id=push_to_hub_model_id,
        learning_rate = 2e-4,
        resume_from_checkpoint=resume_from_checkpoint,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = logging_steps,
        optim = optim,
        weight_decay = weight_decay,
        save_total_limit=save_total_limit,
        save_steps=save_steps,
        lr_scheduler_type = lr_scheduler_type,
        seed = seed,

        output_dir=output_dir,
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1778: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case damerajee/Open-hathi-instruct-1poch).
  warnings.warn(


Map:   0%|          | 0/55215 [00:00<?, ? examples/s]

# Some-cool-information

1. First,you can easily run all teh code and try it yourselves
2. the max memory taken before training was 4.3 gb and after was almost 8gb


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
4.359 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train("checkpoint-path")